In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import random
import re
import os
import datetime
import pandas as pd
import numpy as np

os.chdir('C:/Users/simjh96/Documents/Car_Accident/data/crawling/GIS')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def _get_chromebrowser():
    print("시스템>>> 현재 디렉토리: {}".format(os.getcwd()))
    print("시스템>>> 디렉토리가 맞다면(Y) 아니면(N): ")
    inp = input()
    if inp == 'N':
        print("시스템>>> 크롬드라이버가 있는 위치 입력...")
        os.chdir(input())
    browser = webdriver.Chrome("./chromedriver.exe")
    #self 로 바꿔 arg 에도 넣고
    print("시스템>>> 크롬드라이버를 실행 합니다...")
    return browser

In [ ]:
#in page
browser = _get_chromebrowser()
browser.get('http://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN')


In [ ]:
#경기도 오산시 어린이 사고 수동 설정 후 실행
browser.find_element_by_link_text('검색').click()
time.sleep(5)

In [ ]:


#totalcount
print('시스템>>> total count를 셉니다...')
soup = BeautifulSoup(browser.page_source, "lxml")
tot = soup.find(attrs='total-count')
tot = int(re.findall(r'[0-9]+',tot.text)[0])
print('시스템>>> 총 {}건을 검색합니다'.format(tot))

#저장될 cords
cords = []
    
#1계층
#페이지 넘어가게 loop로 묶어
for i in range(tot//10 + 1):
    print('시스템>>> {} 페이지로 넘어갑니다.'.format(str(i+1)))   
    browser.find_element_by_partial_link_text('{}'.format(str(i+1))).click()    
    
    #2계층
    # 10 건별 클릭
    elms = browser.find_elements_by_class_name('iStyle')
    for j in range(len(elms)):
        time.sleep(np.random.uniform(0.5,1))
        print('시스템>>> {}번째 건을 추출합니다...'.format(j+1))
        elms[j].click()
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'pop-headerN')))
        
        #BBOX 추출
        #수정필요>>>첫 BBOX 가 검색전 메인 화면 좌표가 나와서 좌표가 한칸씩 밀림
        print('시스템>>> BBOX 좌표를 구합니다...')
        soup = BeautifulSoup(browser.page_source, "lxml")
        imgs = soup.find_all(attrs='olTileImage')
        for img in imgs:
            if ('BBOX' in img['src']):
                full_cord = re.findall(r'BBOX=[0-9.,]*&',img['src'])[0][5:-1]
                cord_list = re.findall('[0-9]+.[0-9]+',full_cord)
                cords.append(((float(cord_list[0])+float(cord_list[2]))/2, 
                              (float(cord_list[1])+float(cord_list[3]))/2,
                              soup.find(attrs='td1').text))
                break

cords_df = pd.DataFrame(cords)
with open("{}.csv".format(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M')), "w", encoding="utf8") as f:
    f.write(cords_df.to_csv())

print('시스템>>> 모든 수집을 완료했습니다!')

In [ ]:
#merge all
cords_df = pd.read_csv('2017-2019.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]
cords_df.columns = ['x','y']

b_df = pd.read_csv('2017-2019_b.csv')

df = pd.concat((b_df,cords_df),axis=1)
#=========================================
cords_df = pd.read_csv('2014-2016.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]
cords_df.columns = ['x','y']

b_df = pd.read_csv('2014-2016_b.csv')

df = pd.concat((df,pd.concat((b_df,cords_df),axis=1)))

cords_df = pd.read_csv('2011-2013.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]
cords_df.columns = ['x','y']

b_df = pd.read_csv('2011-2013_b.csv')

df = pd.concat((df,pd.concat((b_df,cords_df),axis=1)))

cords_df = pd.read_csv('2008-2010.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]
cords_df.columns = ['x','y']

b_df = pd.read_csv('2008-2010_b.csv')

df = pd.concat((df,pd.concat((b_df,cords_df),axis=1)))

cords_df = pd.read_csv('2007.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]
cords_df.columns = ['x','y']

b_df = pd.read_csv('2007_b.csv')

df = pd.concat((df,pd.concat((b_df,cords_df),axis=1)))


with open("{}_concated.csv".format(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M')), "w", encoding="utf8") as f:
    f.write(df.to_csv())